In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
1888,Euroclear: лед тронулся Международный депозит...,36-3,clsorg
4832,🇷🇺#GMKN Росатом и Норникель изучат возможнос...,53-3,clsorg
167,"""ВТБ кстати говоря, пока суть да дело решил не...",99-3,clsorg
6208,💼 Народный портфель. Частные инвесторы покупаю...,150-4,clsorg
3383,Центр генетики и репродуктивной медицины Genet...,90-3,clsorg
4907,🇷🇺#MAGN ИНВЕСТПРОГРАММА ММК В 2023 Г СОСТАВИТ...,90-4;90-2,clsorg
6895,🟢 Новости к этому часу ⚪️ Организация эконом...,209-2,clsorg
832,"""🇷🇺#SIBN """"Газпром нефть"""" приступила к развит...",47-4,clsorg
3208,Сильный отчет «Полиметалла» за 1-й квартал 202...,235-4,clsorg
210,"""ЕСЛИ ВЫ ПРОПУСТИЛИ. ИНТЕРЕСНЫЕ ПОСТЫ ЗА ПОСЛЕ...",265-3,clsorg


In [ ]:
m_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
import torch
from transformers import T5ForConditionalGeneration

# m_name = "t5-small"
# trained_model = T5ForConditionalGeneration.from_pretrained(m_name, return_dict=True)
# state_dict = torch.load("./checkpoints/ner-v5.ckpt")["state_dict"]
# state_dict = {k.partition("model.")[2]: v for k, v in state_dict.items()}
# trained_model.load_state_dict(state_dict)
# trained_model.save_pretrained("./pretrained")

In [ ]:
m_name = "./pretrained"
trained_model = T5ForConditionalGeneration.from_pretrained(
    m_name, return_dict=True, torch_dtype=torch.float32
)
trained_model.cuda();

In [ ]:
import onnxruntime as rt
from optimum.onnxruntime import ORTModelForSeq2SeqLM

print(rt.get_available_providers())

ort_model = ORTModelForSeq2SeqLM.from_pretrained(
    "./pretrained",
    provider="CUDAExecutionProvider",
    export=True,
)

In [ ]:
import torch

from src.t5.utils import evaluate_metric, generate_answer_batched

In [ ]:
# [pin]
with torch.inference_mode():
    predictions = generate_answer_batched(
        trained_model=ort_model,
        tokenizer=tokenizer,
        data=test_df[:1200],
        batch_size=64,
        n_beams=1,
        max_length=386,
    )

  0%|          | 0/19 [00:00<?, ?it/s]

/home/worker/workspace/hakaton-gagarin-sentiment_interface/.conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [ ]:
# [pin]
with torch.inference_mode(), torch.cuda.amp.autocast():
    predictions = generate_answer_batched(
        trained_model=trained_model,
        tokenizer=tokenizer,
        data=test_df[:1200],
        batch_size=64,
        n_beams=1,
        max_length=386,
    )

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
ldf = test_df.copy()[:1200]
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 58.241360011302056, 'f1': 0.6373272002260412, 'accuracy': 0.5275}